# CORD 19 Task 2: Population Studies

![](http://cdn.pixabay.com/photo/2020/03/17/22/29/coronavirus-4942077_1280.jpg)

The CORD19 challenge is an excellent proving ground for modern natural language processing. The challenge: to build machine-generated summaries of the Covid 19 research literature to assist in further research. The task is atypical of the average kaggle competition, as there are no training examples (other than a handful of example summaries) against which to train a supervised model, and no holdout test set against which to score submissions. Competitors must therefore develop unsupervised methods to generate the target output and define their own means of evaluating this output. This setting is typical of the broader challenges that nlp practitioners face "in the wild": choosing a latent space capable of representing the seemingly limitless variability of language data that fits a given task, and similarly designing an output space that meets the task specification.

## contents

1. [Approach](#Approach)
2. [Results](#Results)
3. [Pros](#Pros)
4. [Cons](#Cons)
5. [Next Steps](#Next-Steps)
6. [Notes on Study Design](#Notes-on-Study-Design)
7. [Example - Combatting Resource Failures](#Example---Combatting-Resource-Failures)

# Approach

This notebook describes a solution capable of generating the summary tables inf the format specified for the task. The approach is divided into search and summary stages, with a separate mdoel / tool for both of these stages:

### Search Tool:

The search tool identifies papers that answer a given research question in two steps:

1. **Regex search**: Articles of interest are identified by a simple regular expression keyword search of the article abstracts
2. **Sentence-level TandA QA**: Abstracts identified in stage one are divided into sentences, and these sentences are scored based on the likelihood that they answer a generalised research question. Logits generated by the [Amazon Alexa Team's RoBERTa-based TandA model](https://github.com/alexa/wqa_tanda) are used to define these scores.  Those papers whose abstracts contain scores above a certain threshold are deemed to answer the research question.

### Summary Tool:

Full text from each article identified in stage one is taken and analysed for the following information:

* **Study Design**: regular expressions highlight explicit mentions of a specific study design in the text, and other features that may relate to a given study design, for example statistical analysis terms (e.g. "p<", "ci") or the mention of modelling methods (e.g. "regresion", "SVM"). These are used to determine the most likely study design.
* **Measure of Evidence**: [Spacy](https://spacy.io/)'s POS and NER tools are used to highlight word tokens relating to quantities of study subjects (e.g. "patients", "participants", "samples") or study locations mentioned in the text. Tokens identified in this process are cross-referenced with their overall frequency in the text; the quantities of tokens demed significant are returned as measures of evidence (e.g. "100 patients", "1000 participants").
* **Challenge**: Similar to the second stage of the search process, a TandA question-answering model is used to score the likelihood that each sentence in the abstract text answers the question **"what is the problem / issue / challenge?"** (this can be customised by the user) - the highest scoring sentence is used as the **challenge** feature
* **Solution**: Again, sentences from the full text of the paper are scored based on the likelihood they answer a question, which defaults to **"what action should be taken?"** - the highest scoring sentence is used as the **solution** feature.
* **Addressed Population**: regular expressions are used to identify keywords relating to gropus (e.g. "workers", "patients", "groups") - POS tagging rules are then used to parse descriptions from the word tokens immediately following or preceeding these keywords to identify population descriptions (e.g. "**mental health** workers...", "patients **with diabetes**"). The most common of the group descriptions are identified as addressed populations.

# Results

The results of the search / summarization process can be seen in each of the csv files in the output to this notebook. All have been prepared within the kaggle notebook environment. An example of search process is featured below, that produces the summary table for "RESEARCH QUESTION", with comments on the output for that task and a more detailed explanation of the search and summary results. Scripts used to build the summary tables for the other research questions can be found in the following notebooks:

* [Modes of communicating with target high-risk populations (elderly, health care workers)](https://www.kaggle.com/dustyturner/modes-of-communicating-with-target-communities)
* [Management of patients who are underhoused or otherwise lower socioeconomic status](https://www.kaggle.com/dustyturner/management-of-patients-who-are-underhoused)
* [What are ways to create hospital infrastructure to prevent nosocomial outbreaks and protect uninfected patients?](https://www.kaggle.com/dustyturner/hospital-infrastructure-to-prevent-outbreaks)
* [Methods to control the spread in communities, barriers to compliance](https://www.kaggle.com/dustyturner/methods-to-control-spread-in-communities)

This notebook and the above can all be forked and run to reproduce the output results. The author's [tanda_search_qa_tool](https://github.com/samrelins/tanda_search_qa_tool) repository contains the code for the search and summary tools as well as various helper functions for preprocessing of the metadata and document texts.

**Please Note:** The results provided are an attempt to trade off specificity and accuracy of the summaries with quantity, and to demonstrate the results of this method un altered / curated. With a greater number of queries and less specific search criteria, many more potentially useful studies can be identified and included with these results.


# Pros

* The approach works!: It is capable of parsing the entire CORD dataset to highlight the information required in the summary tables
* Simple methodology: each stage of the task is based on a process that can be easily understood and explained to lay-people
* Flexible: the different stages of the search and summary process allow for considerable flexibility, meaning results can be tailored to a wide range of research questions, beyond that of the current task specification
* Visible: the tools are capable of producing visual cues to assist in refining the search process and explaining the results
* Potential for fine-tuning: the TandA model is designed to be easily adapted to new tasks with minimal labled data. The simplicity of the sentence-level QA task offers the potential to produce labled training data to fine-tune the model and better recognise features of the covid-19 literature

# Cons

* Computationally intensive: the question answering elements of the search and summary process require individual sentences to be analysed by a RoBERTa base model and token-level POS and NER tagging for each paper in the summary tables. As a result, the search process is slow and would not be suitable as an on-demand search-engine based solution like a number of the task 1 submissions
* Reliant on CORD dataset: the summary tool is unable to produce results for papers that are not featured in the json parses provided in the competition data. As such, a significant portion of promising results from the search process cannot be summarised and are not included in the results. Inaccuracies in the parsing can also cause some unusual features in the output tables.
* Reliant on user input: producing summaries is still heavily reliant on user input to specify keywords, search questions that identify relevant publications, and parameters to define features in the summary data. In general, the model can't be thought of as independently generating the summaries


# Next Steps

As highlighted above, the TandA model was conceived as an answer to training question answering models in situations where task-specific training data is scarce. As such, this approach has potential for significant improvement by producing labled question / answer sentence pair examples from the covid-19 literature. Given that sentence-level QA is a binary classification task, it is relatively easy to procedurally generate training examples that can be used to fine-tune the model. This approach offers potentially large improvements to the quality of the search results.

Whilst developing this solution, attempts were made to use the summary examples provided in the competition data as training data, aiming to develop classification models to identify the study design and important features from publications. The small number of examples, and inconsistencies in the labeling prevented this from being a successful approach. As greater quantities of quality paper summaries are produced, this may become a viable means to parse information from the papers that has a greater basis on language understanding, and is less reliant on keyword tagging.

# Notes on Study Design

Significant time was spent approaching the problem of identifying study design; so much so that it merits special attention. 

The author is not an epidemiologist, nor has any significant scientific research experience, and so is well versed in the specifics of medical research design. Given this, the example summaries and descriptions of the study designs were heavily relied upon when designing this part of the summary tool. This presented some considerable confusion given inconsistencies between assigned labels, the descriptions, and in several cases with the author's own description of their study design! Also, the descriptions did not seem to fit with research areas other than medicine / epidemiology - good examples were papers relating to biochemistry or molecular biology primary studies.

In an effort to simplify matters the following decisions were taken:

* General Rule: the more specific study designs - **systematic review and meta-analysis, prospective / retrospective observational studies, cross sectional studies, and case series** -  were only assigned to a paper if the authors explicitly stated their study design
* Expert Review: The description of the "expert review" study type states "provides quantitative secondary data" - this was contrary to the majority of such studies in the summary tables that included no such quantitative data. Moreover, the descriptions of the study designs did not allow for studies that were not a structured literature review, but did not include any numerical analysis. On this basis, a decision was taken to follow the examples in the summary tables and label non-numerate studies without a structured literature review component as "expert review"
* Analytical Studies: a large number of papers in the corpora were clearly quantitative primary studies, with an experimental method, quantitative results and analysis, but did not fit any of the descriptions of quantitative studies. Given the authors did not specify any of the prescribed study designs, it did not seem appropriate to "force a square peg into a round hole" so to speak. As such, studies with these features were labled "analytical study"

It should be stressed that these decisions were taken based on intuition and can easily be adjusted with more specific feedback and guidance.

# Example - Combatting Resource Failures

The following is a quick walkthrough of the usage and features of the search and summarizer tools, answering the research question **What are recommendations for combating/overcoming resource failures?**

In [ ]:
!wget https://github.com/samrelins/tanda_search_qa_tool/archive/master.zip
!unzip master.zip
!wget https://wqa-public.s3.amazonaws.com/tanda-aaai-2020/models/tanda_roberta_base_asnq.tar
%mkdir tanda_roberta_base_asnq
!tar xvf tanda_roberta_base_asnq.tar
%mv /kaggle/working/models /kaggle/working/tanda_roberta_base_asnq/
%mv /kaggle/working/tanda_search_qa_tool-master/* /kaggle/working/
%rm -rf tanda_roberta_base_asnq.tar master.zip

In [ ]:
import numpy as np
import pandas as pd 
from cord_search_qa_tool import CordSearchQATool
from cord_result_summarizer import CordResultSummarizer
from summarizer_helpers import *
from prep_metadata import add_missing_abstracts
from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth', None)

First, the metadata is pre-processed. Papers without abstracts are identified and the first 1500 or more characters from the full text of the paper (if available) are used as an abstract.

In [ ]:
data_dir = "/kaggle/input/CORD-19-research-challenge/"
meta = add_missing_abstracts(data_dir)

Next, the search tool is initialised with the pre-processed metadata and the location of the directory containing the pre-trained Roberta-TandA model. The tool's init process....

1. Filters the metadata to identify papers that relate to covid-19 using a regex search to identify covid-related keywords (e.g. "covid-19", "sars-cov-2", "wuhan coronavirus"). The abstract text and cord_uid of these papers are then stored as key-value pairs in a dictionary which is set as the search tool's `.texts` attribute
2. Initialises the pytorch Roberta model, loading the tokenizer and model into memory and the model to the gpu if required.

In [ ]:
searchtool = CordSearchQATool(covid_meta = meta,
                              qa_model_dir = "tanda_roberta_base_asnq")

The search tool is then ready to use. The aim of the search process can be summarised by two broad strategies:

1. Narrow the search space to relevant papers by searching for keywords
2. Identify papers that answer a specific research question by highlighting sentences that are likely to contain a relevant answer

Beginning with the keyword search, the aim is to find all the papers in the corpus that may relate to the research question; in this example, this is the papers relating to resource shortages. This is done using the search tool's `.search` method. The user defines a list of regex terms that are passed to the `.search` method's `containing` and / or `not_containing` arguments. The `containing_threshold` argument can also be used to set a minimum number of keywords an abstract should contain. The abstracts that meet these criteria are then stored as a list of ids in the search tool's `.search_results` attribute under the `search_name` provided (or a default name if none is provided). The regex search uses a basic logical `OR` criteria, so any abstracts containing a sufficient number of any of the keywords (individually or in combination) will be returned.

The search tool is also able to provide a html output of the search results with the `return_html_search_results` method, which is helpful in visualising the progress of any searches:

In [ ]:
# search for keywords relating to resource shortages
searchtool.search(search_name="shortage", 
               containing=["shortage", "resources", "supply\W",
                           "availab", "scarce"],
               containing_threshold=2)

# display the first 3 results of the "shortage" search above
display(HTML(
    searchtool.return_html_search_results(search_name="shortage",
                                          n_results=3)
))

__________________________________________________________

Once satisfied that the search contains studies that may relate to the research question, papers that contain answers relevant to the research question can then be identified. 

The `return_answers` method uses the RoBERTa TandA model to output logit scores, indicating the likelihood that each sentence of an abstract answers a given question  - the higher the score, the more likely the model "thinks" the sentence contains an answer to the question posed. The user defines the search (i.e. the above "shortage" search) from which the abstracts are taken. The method takes the following arguments:

* **question**: the question all sentences are to be scored against
* **search_name**: the name of a search (like that above), from which the abstracts are taken and scored. If `None` every abstract will be used (not advisable as this will be a huge set of sentences over which to search)
* **min_score**: the minimum score a sentence must have to be returned in the output. If `None` the method will return a score for every sentence.
* **max_length**: the maximum length of a sentence in words / token. This is a variable used by the Roberta model to define the fixed-length of the model inputs and defaults to 128.

The output is a list of tuples with the format `(cord_uid, sentence index, sentence text, logit score)`, ordered by descending score. For example, take the following: 

In [ ]:
# collect answers to the question specified
eg_answers = searchtool.return_answers(
    search_name="shortage", 
    question="preventing or stopping shortages or supply failures",
    min_score=-2)
eg_answers[0]

The above is the highest scoring sentence to the question **"preventing or stopping shortages or supply failures"** from the abstracts identified in the **"shortage"** search. The output shows the cord_uid of the paper, the order in which that sentence appears in the abstract, the sentence text and finally the score. 

To make visualising the QA search prcess easier, the tool also has a `return_html_answers` method. This works in exactly the same way as the `return_answers` method, featuring  the same arguments and outputting the same list of tuples. An additional HTML output is provided alongside the tuples, that features the title and full abstract from the highest scoring papers (the number of which can be set with the `top_n` argument), whith sentences above a certain score highlighted (specified by the `highlight_score` argument):

In [ ]:
# same output as above, plus html to visualise the top 5 answers
answers_1, html_answers = searchtool.return_html_answers(
    search_name="shortage", 
    question="preventing or stopping shortages or supply failures",
    min_score=-2,
    top_n = 5,
    highlight_score=-2)

display(HTML(html_answers))

______________________________________________

Analysis of the output shows some of the papers relate to "hearing loss" without any mention of resourcing issues, which is likely a confusion of the concept of "shortage" by the model given the question. Such papers can be removed from the search process using the `refine_search` method. This takes an existing searche and applies a new set of criteria to the results of the specified search (rather than the whole corpora in the metadata). The arguments of the `refine_search` method are the same as the `search` method. The following removes the papers relating to hearing loss from the original results of the "shortage" search:

In [ ]:
searchtool.refine_search(search_name="shortage",
                         not_containing=["hearing loss", "loss of hearing"])

Note that, by printing the `search_results` attribute of the search tool, a description of each search and subsequent refined search can be seen:

In [ ]:
searchtool.search_results

And with the refined search results, the `return_answers` method can again be used to retrieve relevant papers:

In [ ]:
# run the search again with the excluded papers
answers_1 = searchtool.return_answers(
    search_name="shortage", 
    question="preventing or stopping shortages or supply failures",
    min_score=-2)

*The full script relating to the covid search tool can be found in the [cord_search_qa_tool.py](https://github.com/samrelins/tanda_search_qa_tool/blob/master/cord_search_qa_tool.py) and [tanda_search_qa_tool.py](https://github.com/samrelins/tanda_search_qa_tool/blob/master/cord_search_qa_tool.py) modules in the task [github repo](https://github.com/samrelins/tanda_search_qa_tool)*

Once a list of satisfactory answers has been achieved, the second stage of the process can begin. A summarizer tool takes the unique ids from each of the papers highlighted in the search process, and automatically summarises them to produce a dataframe in the format of the competition specification. First, a list of cord_uids for the answers identified above is required:

In [ ]:
#return a list of individual ids for each paper containing an answer / answers
cord_uids_1 = []
for cord_uid, *_ in answers_1:
    if cord_uid not in cord_uids_1:
        cord_uids_1.append(cord_uid)

Then the summariser tool can be initialised using these ids as the `cord_uids` argument. The tool also requires the metatdata dataframe (the `meta` argument), the directory containing the Roberta-Tanda model (the `tanda_dir` argument), and the location of the CORD-19 dataset (the `data_dir` argument) to access the full text of each paper:

In [ ]:
summarizer_1 = CordResultSummarizer(cord_uids=cord_uids_1,
                                          meta=meta,
                                          data_dir=data_dir,
                                          tanda_dir="tanda_roberta_base_asnq")

The summary tool can then output a summary table of these papers by calling the `summary_table` method which takes three arguments:

* **challenge_question**: the question used by the summariser to identify the **challenge** feature for each text. The sentence from each abstract that scores the highest against this feature is used. This defaults to "what is the problem issue challenge"
* **solution_question**: similar to the **challenge_question** this is the question used to identify the solution feature. This defaults to "what action should be taken".
* **n_hits**: This is the number of times a particular group must be mentioned in the text to be used in the **addressed_population** feature. This defaults to 2.

The following is the summary output for the "shortage" results:

In [ ]:
summary_table_1 = summarizer_1.summary_table(
    challenge_question="what is the problem issue challenge", # these are defaults and can be left out as below
    solution_question="what action should be taken" 
)

display_features = ["study", "addressed_population", "strength_of_evidence", 
                    "study_type", "challenge", "solution", "journal"]
summary_table_1[display_features].head(10)

The summary process takes place in the following stages:

* **study design**: regular expression keywords are used to identify sections of the text where the author specifies their study design. If these are detected the study design is allocated accordingly. If not, keywords highlighting phrases commonly found in simulation studies are identified that can be confidently used to assign that study type. Remaining studies are queried for keywords relating to quantitative and experimental study designs - these are classified as "analytical studies" to differentiate them from the non-primary literature review type studies. The remaining studies are then classified as "editorial" if they are addressed "to the editor" or "expert review" otherwise.

* **challenge / solution**: These sections make use of the same QA model as the search tool. Default questions **"what is the problem / issue / challenge?"** and **"what action should be taken?"** (minus the punctuation) are used to rank sentences from the paper text. The highest scoring answers are used in the respective sections as a high level summary. The challenge question uses only text from the abstract, as the abstracts are generally found to be a better summary of the issues the paper addresses - the solutions, however, are often better found in the full text. The tool does allow for custom challenge / solution questions should that be required, although the author has found the default questions to produce the best results in most cases.

* **addressed population**: The tool starts by identifying generic population terms i.e. "people", "patients", "staff", "workers" via regular expression search. The [Spacy](https://spacy.io/) POS tags from the words immediately preceding and following these terms are then examined - particular grammatical constructs are looked for that identify descriptions of groups, for example "critically ill patients" or "people experiencing economic hardship". Terms that are found multiple times in the text, determined by a user-defined threshold, are then returned as the addressed populations.

* **measure of evidence**: This also makes use of Spacy POS tags along with NER tagging. The named entities in the text are iterated through to identify cardinal numbers that relate to evidential quantities, such as"respondents", "patients", "studies". These are then compared with the overall mentions of these terms within the text to ensure relevance, and then reported as measures of evidence. Care is taken to eliminate numbers that form part of the referencing or structure of the text, and are not themselves quantities. 


The remaining features can easily be parsed from the metadata provided in the task dataset. All of the code and logic for the above can be found in the [cord_result_summarizer.py](https://github.com/samrelins/tanda_search_qa_tool/blob/master/cord_result_summarizer.py) and [summarizer_helpers.py](https://github.com/samrelins/tanda_search_qa_tool/blob/master/summarizer_helpers.py) files in the [competition repo](https://github.com/samrelins/tanda_search_qa_tool). 

The TandA model is very sensitive to the wording of the questions it is provided. The best approach to the search process is generally to ask a number of related questions, which enlarges the search space for potential answers in an iterative manner. This is done in the following cells, which return answers to other questions relating to resource failures, resulting in the final output table. It should be noted that none of the outputs submitted are exhaustive and this process has potential to identify and summarise many more useful studies. 

(Unhide the outputs below to see the results from each subsequent search and summary)

In [ ]:
answers_2, html_answers = searchtool.return_html_answers(
    search_name="shortage", 
    question="reducing demand or use of scarce low resources",
    min_score=-2,
    highlight_score=-2)

display(HTML(html_answers))

In [ ]:
cord_uids_2 = []
for cord_uid, *_ in answers_2:
    if cord_uid not in cord_uids_2 and cord_uid not in cord_uids_1:
        cord_uids_2.append(cord_uid)
        
summarizer_2 = CordResultSummarizer(cord_uids=cord_uids_2,
                                          meta=meta,
                                          data_dir=data_dir,
                                          tanda_dir="tanda_roberta_base_asnq")

summary_table_2 = summarizer_2.summary_table()

summary_table_2[display_features].head(10)

In [ ]:
answers_3, html_answers = searchtool.return_html_answers(
    search_name="shortage", 
    question="allocating or prioritising care or scarce resources",
    min_score=-2,
    highlight_score=-2)

display(HTML(html_answers))

In [ ]:
cord_uids_3 = []
previous_results = cord_uids_1 + cord_uids_2
for cord_uid, *_ in answers_3:
    if cord_uid not in cord_uids_3 and cord_uid not in previous_results:
        cord_uids_3.append(cord_uid)
        
summarizer_3 = CordResultSummarizer(cord_uids=cord_uids_3,
                                          meta=meta,
                                          data_dir=data_dir,
                                          tanda_dir="tanda_roberta_base_asnq")

summary_table_3 = summarizer_3.summary_table()

summary_table_3[display_features].head(10)

In [ ]:
answers_4, html_answers = searchtool.return_html_answers(
    search_name="shortage", 
    question="managing intensive care bed shortage",
    min_score=-2,
    highlight_score=-2)

display(HTML(html_answers))

In [ ]:
cord_uids_4 = []
previous_results = cord_uids_1 + cord_uids_2 + cord_uids_3
for cord_uid, *_ in answers_4:
    if cord_uid not in cord_uids_4 and cord_uid not in previous_results:
        cord_uids_4.append(cord_uid)
        
summarizer_4 = CordResultSummarizer(cord_uids=cord_uids_4,
                                          meta=meta,
                                          data_dir=data_dir,
                                          tanda_dir="tanda_roberta_base_asnq")

summary_table_4 = summarizer_4.summary_table()

summary_table_4[display_features].head(10)

In [ ]:
%rm -rf *

%cp ../input/hospital-infrastructure-to-prevent-outbreaks/what_are_ways_to_create_hospital_infrastructure_to_prevent_nosocomial_outbreaks.csv /kaggle/working
%cp ../input/management-of-patients-who-are-underhoused/management_of_patients_who_are_underhoused_or_otherwise_lower_social_economic_status.csv /kaggle/working
%cp ../input/methods-to-control-spread-in-communities/methods_to_control_the_spread_in_communities.csv /kaggle/working
%cp ../input/modes-of-communicating-with-target-communities/modes_of_communicating_with_target_high_risk_populations.csv /kaggle/working

In [ ]:
summary_table = pd.concat([summary_table_1, 
                           summary_table_2,
                           summary_table_3,
                           summary_table_4])
summary_table.to_csv("what_are_recommendations_for_combating_overcoming_resource_failures.csv")